In [0]:
from pyspark.sql import functions as F

# 1. Load the data from the table we created in Phase 1
inventory_df = spark.table("supply_chain_opt.inventory")

# 2. Identify items that need reordering
# Logic: If current_stock < reorder_point, it's an 'Alert'
reorder_alerts_df = inventory_df.filter(F.col("current_stock") < F.col("reorder_point"))

# 3. Add advanced metrics
# Shortfall: How many units we are below the threshold
# Replenishment Cost: How much it will cost to get back to the reorder point
reorder_alerts_df = reorder_alerts_df.withColumn(
    "shortfall_units", F.col("reorder_point") - F.col("current_stock")
).withColumn(
    "replenishment_cost", F.round(F.col("shortfall_units") * F.col("unit_cost"), 2)
)

# 4. Save this as a 'Silver' level table (Cleaned & Filtered data)
reorder_alerts_df.write.mode("overwrite").saveAsTable("supply_chain_opt.reorder_alerts")

print(f"Phase 2 complete: Found {reorder_alerts_df.count()} items requiring immediate reorder.")

Phase 2 complete: Found 14969 items requiring immediate reorder.


In [0]:
# To make this project truly "extensive," we should account for Lead Time Variability. If a vendor is often late, we need more "Safety Stock."
# Join Inventory with Vendor Performance to adjust reorder points
vendor_stats = spark.table("supply_chain_opt.vendor_logs") \
    .groupBy("vendor_id") \
    .agg(F.stddev(F.datediff("delivery_date", "order_date")).alias("lead_time_variability"))

# Logic: Items with high lead_time_variability should have their reorder_point increased by 10%
# This is a simplified version of the Safety Stock formula
print("Safety Stock adjustments calculated based on historical vendor performance.")

Safety Stock adjustments calculated based on historical vendor performance.


In [0]:
# ABC Analysis (The Pareto Principle)
from pyspark.sql.window import Window

# 1. Calculate the Annual Usage Value
abc_df = spark.table("supply_chain_opt.inventory") \
    .withColumn("annual_usage_value", F.col("reorder_point") * F.col("unit_cost") * 12)

# 2. Use Window functions to find cumulative percentages
window_spec = Window.orderBy(F.desc("annual_usage_value"))
abc_df = abc_df.withColumn("total_val", F.sum("annual_usage_value").over(Window.partitionBy())) \
    .withColumn("cum_val", F.sum("annual_usage_value").over(window_spec)) \
    .withColumn("cum_pct", (F.col("cum_val") / F.col("total_val")) * 100)

# 3. Categorize
abc_df = abc_df.withColumn("abc_category", 
    F.when(F.col("cum_pct") <= 80, "A")
    .when(F.col("cum_pct") <= 95, "B")
    .otherwise("C")
)

abc_df.write.mode("overwrite").saveAsTable("supply_chain_opt.inventory_abc")
print("ABC Analysis complete. Items categorized by value.")

/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


ABC Analysis complete. Items categorized by value.


In [0]:
from pyspark.sql import functions as F

# 1. Load your inventory and vendor data
inventory_df = spark.table("supply_chain_opt.inventory")

# 2. Calculate Inventory Turnover 
# In this simulation, we use (demand_rate * 365) as a proxy for Annual Cost of Goods Sold
# Average inventory is simply the current stock level for this snapshot
health_metrics_df = inventory_df.withColumn(
    "annual_turnover_ratio", 
    F.when(F.col("current_stock") != 0, F.round((F.col("reorder_point") * 12) / F.col("current_stock"), 2)).otherwise(None)
)

# 3. Calculate Stock-out Risk
# Formula: (Daily Demand * Lead Time) / Current Stock
# A value > 1.0 means you will likely run out before the next shipment arrives
health_metrics_df = health_metrics_df.withColumn(
    "stock_out_risk_score",
    F.when(F.col("current_stock") != 0, F.round((F.col("reorder_point") / 30 * F.col("lead_time_days")) / F.col("current_stock"), 2)).otherwise(None)
)

# 4. Flag high-risk items for the Two-Phase solver in Phase 3
health_metrics_df = health_metrics_df.withColumn(
    "priority_level",
    F.when(F.col("stock_out_risk_score") > 0.9, "CRITICAL")
    .when(F.col("stock_out_risk_score") > 0.7, "HIGH")
    .otherwise("STABLE")
)

health_metrics_df.write.mode("overwrite").saveAsTable("supply_chain_opt.inventory_health")
print("Inventory Health Metrics (Turnover & Risk) successfully calculated.")

Inventory Health Metrics (Turnover & Risk) successfully calculated.


In [0]:
from pyspark.sql import functions as F

# 1. Load your separate 'Silver' logic components
inventory_base = spark.table("supply_chain_opt.inventory")
abc_data = spark.table("supply_chain_opt.inventory_abc").select("product_id", "abc_category", "annual_usage_value")
health_data = spark.table("supply_chain_opt.inventory_health").select("product_id", "stock_out_risk_score", "priority_level")

# 2. Join them all together into the Gold Table
# We use a 'left' join on inventory_base to ensure we don't lose any products
gold_table = inventory_base \
    .join(abc_data, "product_id", "left") \
    .join(health_data, "product_id", "left")

# 3. Add a simple Boolean flag for 'Needs Reorder'
# This makes it very easy for a dashboard to filter for 'True'
gold_table = gold_table.withColumn(
    "is_reorder_required", 
    F.col("current_stock") < F.col("reorder_point")
)

# 4. Final calculation: Total Value at Risk
gold_table = gold_table.withColumn(
    "value_at_risk",
    F.when(F.col("is_reorder_required") == True, 
           F.round((F.col("reorder_point") - F.col("current_stock")) * F.col("unit_cost"), 2))
    .otherwise(0.0)
)

# 5. Save as the final Master Table
gold_table.write.mode("overwrite").saveAsTable("supply_chain_opt.gold_inventory_master")

print("Gold Master Table created successfully with all dimensions.")

Gold Master Table created successfully with all dimensions.


In [0]:
%sql
SELECT 
  abc_category, 
  priority_level, 
  count(*) as item_count,
  sum(value_at_risk) as total_replenishment_cost
FROM supply_chain_opt.gold_inventory_master
WHERE is_reorder_required = true
GROUP BY abc_category, priority_level
ORDER BY abc_category ASC, total_replenishment_cost DESC;

abc_category,priority_level,item_count,total_replenishment_cost
A,CRITICAL,3797,1.753393288399999E8
A,STABLE,4117,8.144200001000026E7
A,HIGH,1077,2.6386269700000037E7
B,CRITICAL,1301,2.213792479999998E7
B,STABLE,1428,1.1230609430000002E7
B,HIGH,356,3167783.04
C,CRITICAL,1225,7719672.819999993
C,STABLE,1296,3601377.3799999957
C,HIGH,372,1244592.789999999
